In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/Amazon Fine Food Review/Reviews.csv', delimiter = ',')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
df = df.dropna()
df.reset_index(inplace=True)

In [5]:
X = df['Summary'][:100000]
type(X)

pandas.core.series.Series

In [21]:
Y_ = df['Score'][:100000]
arr = np.empty(100000)
Y = pd.Series(arr)
for i in range (0,100000):
  if Y_[i] >= 3:
    Y[i] = 1
  else:
    Y[i] = 0
type(Y)

pandas.core.series.Series

In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [23]:
voc_size=5000

In [24]:
import nltk
import re
from nltk.corpus import stopwords

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, 100000):
    review = re.sub('[^a-zA-Z]', ' ', X[i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [27]:
corpus[0]

'good qualiti dog food'

In [28]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2418, 3650, 1901, 1393],
 [1252],
 [144, 1258],
 [2056, 2710],
 [1106, 4672],
 [3362, 4672],
 [1106, 2418, 2671, 1944],
 [3478, 3425, 4672],
 [3619, 2837],
 [785, 1901, 1393],
 [3667, 1820, 2727, 2740],
 [1027, 1392, 3765, 1393, 3826, 522, 1393],
 [1027, 295, 2879, 1393],
 [2647, 3763],
 [1827, 591, 1453],
 [1455, 591, 4603],
 [3616, 508],
 [1392],
 [1106, 3005, 1643],
 [1752, 3323, 3530],
 [429, 2647],
 [591],
 [1869, 3358],
 [591],
 [3438, 2980, 2669],
 [591, 1827],
 [1259, 321],
 [1106, 2912, 873],
 [1453],
 [3667, 1820, 2727, 2740],
 [1106, 4692],
 [508],
 [3667, 3501, 3808],
 [2418, 3501],
 [1106, 4090, 3808, 1692],
 [2569],
 [1392, 2915, 3553, 3808],
 [3808],
 [2418, 2422, 4404, 302],
 [3412, 2644, 3034],
 [3010, 3808, 3894, 508, 1106],
 [3808, 3808, 1798],
 [1393, 1106],
 [2418, 1820, 3034],
 [1106, 508, 4852],
 [4218, 3808],
 [2418],
 [1299],
 [2418, 2878, 1439, 3633, 2898, 4707],
 [2051],
 [2373],
 [1820, 2418, 1625, 3676],
 [4458, 3872, 2710, 1412, 1275],
 [2710, 1412],
 [1

In [29]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3650 1901 1393]
 [   0    0    0 ...    0    0 1252]
 [   0    0    0 ...    0  144 1258]
 ...
 [   0    0    0 ...    0    0  358]
 [   0    0    0 ...    0 1453 3727]
 [   0    0    0 ...    0 2418 1961]]


In [30]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 2418, 3650, 1901, 1393], dtype=int32)

In [31]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(Y)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [35]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

Epoch 1/5
1047/1047 [==============================] - 58s 51ms/step - loss: 0.3070 - accuracy: 0.8787 - val_loss: 0.2777 - val_accuracy: 0.8883
Epoch 2/5
1047/1047 [==============================] - 53s 51ms/step - loss: 0.2555 - accuracy: 0.8957 - val_loss: 0.2741 - val_accuracy: 0.8911
Epoch 3/5
1047/1047 [==============================] - 53s 50ms/step - loss: 0.2375 - accuracy: 0.9022 - val_loss: 0.2814 - val_accuracy: 0.8872
Epoch 4/5
1047/1047 [==============================] - 52s 50ms/step - loss: 0.2251 - accuracy: 0.9071 - val_loss: 0.2810 - val_accuracy: 0.8888
Epoch 5/5
1047/1047 [==============================] - 52s 50ms/step - loss: 0.2162 - accuracy: 0.9106 - val_loss: 0.2853 - val_accuracy: 0.8909
